In [1]:
import os
import itertools
import tensorflow as tf
from keras.callbacks import EarlyStopping
from keras import backend as K
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import tools_with_GMP as tools
import plot_confusion_matrix_Copy1 as plot_confusion_matrix
from sklearn.metrics import confusion_matrix
import numpy as np
np.set_printoptions(suppress=True)

training_data, training_label, validation_data, validation_label, validation_cate_label = tools.get_data()


/home/hsiehch/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/hsiehch/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/hsiehch/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/hsiehch/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:5

data: 8528
label: 8528
StratifiedKFold(n_splits=5, random_state=100, shuffle=True)
trian: [   0    1    2 ... 8525 8526 8527] len 6821 test: [  10   12   19 ... 8515 8520 8524] len 1707
trian: [   0    1    2 ... 8524 8525 8527] len 6822 test: [   3    9   15 ... 8509 8521 8526] len 1706
trian: [   0    2    3 ... 8525 8526 8527] len 6822 test: [   1    5    6 ... 8513 8519 8523] len 1706
trian: [   0    1    2 ... 8524 8526 8527] len 6823 test: [   8   11   26 ... 8517 8518 8525] len 1705
trian: [   1    3    5 ... 8524 8525 8526] len 6824 test: [   0    2    4 ... 8512 8522 8527] len 1704


In [3]:
ks = 5
num_layer = 5
bs = 30
lr = 0.001

def create_model(learning_rate, bs, ks, num_layer):
    num_filter = 64
    
    in_data = tf.keras.Input(shape=(None, 1), dtype="float64")
    x = tf.keras.layers.Conv1D(filters = num_filter, kernel_size = ks, activation="relu")(in_data)
#     x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPool1D(2)(x)

    for i in range(2,num_layer+1):
        try:
            if i==num_layer:
                x = tf.keras.layers.Conv1D(filters = num_filter, kernel_size = ks, activation="relu")(x)
                break
            if i%2 != 0:
                num_filter = num_filter *2
            x = tf.keras.layers.Conv1D(filters = num_filter, kernel_size = ks, activation="relu")(x)
            x = tf.keras.layers.MaxPool1D(2)(x)
            if i in [6, 8, 9]:
                x = tf.keras.layers.Dropout(0.5)(x)
        except ValueError:
            print("model overflow[lr, bs, ks, #layer]: ",[learning_rate, bs, ks, i+1])
            return False
    x = tf.keras.layers.GlobalMaxPool1D()(x)
    x = tf.keras.layers.Dense(128, activation = 'relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(32, activation = 'relu')(x)
    outputs = tf.keras.layers.Dense(4, activation = 'softmax')(x)
    
    model = tf.keras.Model(inputs=in_data, outputs=outputs)
    print(model.summary())
    return model

def run(bs, lr, ks, num_layer):
    fold=1
    for index, (X_train, Y_train, X_val, Y_val, val_cat) in enumerate(zip(training_data,
                                                       training_label,
                                                       validation_data,
                                                       validation_label,
                                                       validation_cate_label)):

        X_val, Y_val, val_cat = shuffle(X_val, Y_val, val_cat, random_state=50)
        model = create_model(lr, bs, ks, num_layer)
        optimizer = tf.keras.optimizers.Adam(lr = lr)
        ##early_stop = EarlyStopping(patience=20)
        epochs = 100
        losses = []
        
        for epoch in range(1, epochs+1, 1):
            print("Epoch: {}".format(epoch))
            epoch_losses = []
            
            prog_bar = tf.keras.utils.Progbar(X_train.shape[0])
            prog_bar_val = tf.keras.utils.Progbar(X_val.shape[0])
            train_acc_metric = tf.keras.metrics.Accuracy()
            val_acc_metric = tf.keras.metrics.Accuracy()
            
            X_train, Y_train = shuffle(X_train, Y_train, random_state=0)
            
            for ind, (input_data, input_label) in enumerate(zip(X_train, Y_train)):
                
                with tf.GradientTape() as tape:
                    input_data = input_data.reshape((1, input_data.shape[0], 1))
                    input_label = input_label.reshape((1, input_label.shape[0]))
                    logits = model(input_data)
                    loss_value = tf.keras.losses.categorical_crossentropy(input_label, logits)
                    epoch_losses.append(float(loss_value))
                
                train_acc_metric.update_state(np.argmax(input_label), np.argmax(logits))
                prog_bar.add(1, values=[("train loss", float(loss_value)), ("train accuracy", float(train_acc_metric.result()))])
                
                # update weights
                if (ind+1)%bs == 0:
                    grads = tape.gradient(loss_value, model.trainable_weights)
                    optimizer.apply_gradients(zip(grads, model.trainable_weights))

            avg_epoch_loss = sum(epoch_losses) / (1.0 * len(epoch_losses))
            
            print("{}: {}".format(epoch, avg_epoch_loss))
            losses.append(avg_epoch_loss)
            
            for ind, (input_data, input_label) in enumerate(zip(X_val, Y_val)):
                input_data = input_data.reshape((1, input_data.shape[0], 1))
                input_label = input_label.reshape((1, input_label.shape[0]))
                val_logits = model(input_data)
                val_loss_val = tf.keras.losses.categorical_crossentropy(input_label, val_logits)
                val_acc_metric.update_state(np.argmax(input_label), np.argmax(logits))
                prog_bar_val.add(1, values=[("val loss", float(val_loss_val)), ("val accuracy", float(val_acc_metric.result()))])
        
        print("\n")

#         model.save('model_with_GMP_{}.h5'.format(fold))
#         model = load_model('model_with_GMP.h5')
        
#         evaluation = model.evaluate(x = X_val, y = Y_val)
#         validation_prediction = model.predict_classes(X_val, batch_size=bs)
#         score = f1_score(val_cat, validation_prediction, average=None)
#         print(score)
        
        fold = fold + 1
        
#         test_prediction = model.predict_classes(X_val, batch_size=1)
#         cnf_matrix = confusion_matrix(val_cat, test_prediction)
#         plot_confusion_matrix.plot_confusion_matrix(cnf_matrix, classes=['AF','Noise','Normal','Other'], save_png=True)
        
#         return X_val, val_cat, validation_prediction
    
run(bs, lr, ks, num_layer)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None, 1)]         0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, None, 64)          384       
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, None, 64)          0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, None, 64)          20544     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, None, 64)          0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, None, 128)         41088     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, None, 128)         0   

KeyboardInterrupt: 